In [1]:
# tryout script to do the whole pipeline on the prf data of one sub
# after try to generalize for all tasks 
# put in Nipype format (do mapNodes because then I can iterate throught files, nicer)

In [1]:
# import packages
from nilearn import image, plotting
import matplotlib.pyplot as plt
%matplotlib inline
import os
import glob
import numpy as np
import nibabel as nib
from spynoza.filtering.nodes import Savgol_filter#, Savgol_filter_confounds
from spynoza.conversion.nodes import Percent_signal_change
from nipype import Node, Function
import nipype.pipeline as pe
import nipype.interfaces.io as nio
from bids.grabbids import BIDSLayout

#from nipype.interfaces import fsl
#from nipype.interfaces import freesurfer
#from nipype.interfaces.utility import Function, IdentityInterface


In [2]:
# save relevant params used in json file, 
# afterwards just need to call the field of the list/dict
import json
analysis_params = {}
json_info = open('SBref_analysis_params.json','r').read()
analysis_params.update(json.loads(json_info))

In [3]:
#analysis_params

In [4]:
# define functions
def SGfilt_confound(confounds, tr, polyorder=3, deriv=0, window_length=120):
    import pandas as pd
    from scipy.signal import savgol_filter
    import numpy as np
    import os

    confounds_table = pd.read_csv(confounds, sep='\t', na_values='n/a') #added this line to original spynoza func because it was giving error when str transformed to float
    
    window = np.int(window_length / tr)

    # Window must be odd
    if window % 2 == 0:
        window += 1

    confounds_filt = savgol_filter(confounds_table, window_length=window, polyorder=polyorder,
                              deriv=deriv, axis=0, mode='nearest')

    new_name = os.path.basename(confounds).split('.')[:-1][0] + '_sg.tsv'
    out_file = os.path.abspath(new_name)

    confounds_table = np.asarray(confounds_table).astype(np.float64) # needed to convert to float to support next operation
    pd.DataFrame(confounds_table - confounds_filt).to_csv(out_file, sep='\t', index=False)

    return out_file

In [5]:
# define paths and variables
sub_list = ['5']
ses_list = ['1']
task='soma'

source_pth = '/home/neuro/projects/data/derivatives/fmriprep/'
dest_pth = '/home/neuro/projects/data/derivatives/post_fmriprep/'
sub_num = str(sub_list[0]).zfill(2)
ses_num = str(ses_list[0]).zfill(2)

In [6]:
#gets preprocessed niftis from derivative folder
layout = BIDSLayout(source_pth)
funcdata = layout.get(subject=sub_num,task=task,extensions='space-T1w_preproc.nii.gz',return_type='file') # list of paths to functionals for this sub
confdata = layout.get(subject=sub_num,task=task,extensions='.tsv',return_type='file')  # list of paths to confounds for this sub

In [7]:
funcdata

['/home/neuro/projects/data/derivatives/fmriprep/sub-05/ses-01/func/sub-05_ses-01_task-soma_run-01_bold_space-T1w_preproc.nii.gz',
 '/home/neuro/projects/data/derivatives/fmriprep/sub-05/ses-01/func/sub-05_ses-01_task-soma_run-03_bold_space-T1w_preproc.nii.gz',
 '/home/neuro/projects/data/derivatives/fmriprep/sub-05/ses-01/func/sub-05_ses-01_task-soma_run-04_bold_space-T1w_preproc.nii.gz']

In [8]:
# do some HP filtering (Savitzky–Golay) on bold + confounds
# define Nodes
SG_filter = pe.MapNode(interface=Savgol_filter,
                       name='savgol_filt',
                       iterfield=['in_file'])
SG_filter_confounds = pe.MapNode(Function(function=SGfilt_confound),
                       name='sgfilt_confound',
                       iterfield=['confounds']) #iterfied needs different name because of temp dir

In [9]:
# define settings, right ones? 
SG_filter.inputs.polyorder = SG_filter_confounds.inputs.polyorder = analysis_params['sgfilter_polyorder']
SG_filter.inputs.deriv = SG_filter_confounds.inputs.deriv = analysis_params['sgfilter_deriv']
SG_filter.inputs.window_length = SG_filter_confounds.inputs.window_length = analysis_params['sgfilter_window_length']

if (sub_num=='01' or sub_num=='03') and ses_num=='01': # exception for some initial subjects' sessions
    SG_filter.inputs.tr = SG_filter_confounds.inputs.tr = 1.5
else:
    SG_filter.inputs.tr = SG_filter_confounds.inputs.tr = analysis_params['TR'] 

SG_filter.inputs.in_file = funcdata
SG_filter_confounds.inputs.confounds = confdata


In [10]:
# run and print outputs
res_SG_filter = SG_filter.run()
#res_SG_filter.outputs.out_file

190123-15:03:58,123 nipype.workflow INFO:
	 [Node] Setting-up "savgol_filt" in "/tmp/tmp8edefq9j/savgol_filt".
190123-15:03:58,130 nipype.workflow INFO:
	 [Node] Setting-up "_savgol_filt0" in "/tmp/tmp8edefq9j/savgol_filt/mapflow/_savgol_filt0".
190123-15:03:58,133 nipype.workflow INFO:
	 [Node] Running "_savgol_filt0" ("nipype.interfaces.utility.wrappers.Function")
190123-15:04:25,44 nipype.workflow INFO:
	 [Node] Finished "_savgol_filt0".
190123-15:04:25,49 nipype.workflow INFO:
	 [Node] Setting-up "_savgol_filt1" in "/tmp/tmp8edefq9j/savgol_filt/mapflow/_savgol_filt1".
190123-15:04:25,53 nipype.workflow INFO:
	 [Node] Running "_savgol_filt1" ("nipype.interfaces.utility.wrappers.Function")
190123-15:04:51,448 nipype.workflow INFO:
	 [Node] Finished "_savgol_filt1".
190123-15:04:51,452 nipype.workflow INFO:
	 [Node] Setting-up "_savgol_filt2" in "/tmp/tmp8edefq9j/savgol_filt/mapflow/_savgol_filt2".
190123-15:04:51,457 nipype.workflow INFO:
	 [Node] Running "_savgol_filt2" ("nipype.int

In [11]:
res_SG_filter_confounds = SG_filter_confounds.run()
#res_SG_filter_confounds.outputs.out

190123-15:05:26,414 nipype.workflow INFO:
	 [Node] Setting-up "sgfilt_confound" in "/tmp/tmp__m8kd2s/sgfilt_confound".
190123-15:05:26,419 nipype.workflow INFO:
	 [Node] Setting-up "_sgfilt_confound0" in "/tmp/tmp__m8kd2s/sgfilt_confound/mapflow/_sgfilt_confound0".
190123-15:05:26,423 nipype.workflow INFO:
	 [Node] Running "_sgfilt_confound0" ("nipype.interfaces.utility.wrappers.Function")
190123-15:05:26,452 nipype.workflow INFO:
	 [Node] Finished "_sgfilt_confound0".
190123-15:05:26,454 nipype.workflow INFO:
	 [Node] Setting-up "_sgfilt_confound1" in "/tmp/tmp__m8kd2s/sgfilt_confound/mapflow/_sgfilt_confound1".
190123-15:05:26,457 nipype.workflow INFO:
	 [Node] Running "_sgfilt_confound1" ("nipype.interfaces.utility.wrappers.Function")
190123-15:05:26,487 nipype.workflow INFO:
	 [Node] Finished "_sgfilt_confound1".
190123-15:05:26,489 nipype.workflow INFO:
	 [Node] Setting-up "_sgfilt_confound2" in "/tmp/tmp__m8kd2s/sgfilt_confound/mapflow/_sgfilt_confound2".
190123-15:05:26,492 nipy

In [12]:
#Convert data to percent signal change
#define Nodes
psc = pe.MapNode(interface=Percent_signal_change, 
              name='percent_signal_change',
             iterfield = ['in_file'])

In [13]:
# define settings
psc.inputs.func = 'median'
psc.inputs.in_file = res_SG_filter.outputs.out_file


In [14]:
# run and print outputs
res_psc = psc.run()
#res_psc.outputs.out_file

190123-15:05:33,806 nipype.workflow INFO:
	 [Node] Setting-up "percent_signal_change" in "/tmp/tmpqerd1kmk/percent_signal_change".
190123-15:05:33,813 nipype.workflow INFO:
	 [Node] Setting-up "_percent_signal_change0" in "/tmp/tmpqerd1kmk/percent_signal_change/mapflow/_percent_signal_change0".
190123-15:05:33,817 nipype.workflow INFO:
	 [Node] Running "_percent_signal_change0" ("nipype.interfaces.utility.wrappers.Function")


<string>:37: RuntimeWarning: invalid value encountered in true_divide


190123-15:05:57,142 nipype.workflow INFO:
	 [Node] Finished "_percent_signal_change0".
190123-15:05:57,144 nipype.workflow INFO:
	 [Node] Setting-up "_percent_signal_change1" in "/tmp/tmpqerd1kmk/percent_signal_change/mapflow/_percent_signal_change1".
190123-15:05:57,147 nipype.workflow INFO:
	 [Node] Running "_percent_signal_change1" ("nipype.interfaces.utility.wrappers.Function")


<string>:37: RuntimeWarning: invalid value encountered in true_divide


190123-15:06:19,54 nipype.workflow INFO:
	 [Node] Finished "_percent_signal_change1".
190123-15:06:19,56 nipype.workflow INFO:
	 [Node] Setting-up "_percent_signal_change2" in "/tmp/tmpqerd1kmk/percent_signal_change/mapflow/_percent_signal_change2".
190123-15:06:19,59 nipype.workflow INFO:
	 [Node] Running "_percent_signal_change2" ("nipype.interfaces.utility.wrappers.Function")


<string>:37: RuntimeWarning: invalid value encountered in true_divide


190123-15:06:40,936 nipype.workflow INFO:
	 [Node] Finished "_percent_signal_change2".
190123-15:06:40,941 nipype.workflow INFO:
	 [Node] Finished "percent_signal_change".


In [15]:
def nistats_confound_pca_glm(nifti_file, confounds_file,output_pth):# which_confounds,output_pth):    # function adapted from utils.py in pearl_7T Git
    
    import pandas as pd
    import nibabel as nb
    import numpy as np
    import os
    from nistats.regression import OLSModel
    from scipy.stats import zscore
    from nilearn.image import math_img
    from nilearn.plotting import plot_stat_map
    import matplotlib.pyplot as plt
    from sklearn.decomposition import PCA

    infile = nb.load(nifti_file)
    mean_img = math_img('np.mean(infile, axis=-1)', infile=infile)
    in_data = infile.get_data().astype(np.float32)
    
    confounds_table = pd.read_csv(confounds_file, sep='\t', na_values='n/a')#[which_confounds] #select subgroup of confounds
    
    # Z-SCORING 1ST, THEN OBTAINING COMPONENTS
    # we assume the confounds nor the nifti_file need temporal filtering 
    z_conf = confounds_table.apply(zscore) #do zscore of each value in the sample (scale data matrix)
    
    pca = PCA(0.95,whiten=True) #choose the minimum number of principal components such that at least 95% of the variance is retained.
    pca_confs = pca.fit_transform(np.nan_to_num(z_conf))
    num_comp = pca.n_components_

    all_confs = pd.DataFrame(pca_confs, columns=['comp_{n}'.format(n=n) for n in range(num_comp)])
    all_confs['intercept'] = np.ones(infile.shape[-1]) #add an intercept column filled with ones
    
    om = OLSModel(np.array(all_confs)) #create a least squares model based on nuisances
    om_rr = om.fit(in_data.reshape((-1,infile.shape[-1])).T) #fit bold data to model
    
    resid_img = nb.Nifti1Image(om_rr.resid.T.reshape(infile.shape).astype(np.float32), affine=infile.affine, header=infile.header)
    cleaned_img = math_img('(resid_img + mean_img[...,np.newaxis]).astype(np.float32)', resid_img=resid_img, mean_img=mean_img)
    
    output_nifti = output_pth+os.path.basename(nifti_file).replace('.nii.gz', '_nuis.nii.gz')
    cleaned_img.to_filename(output_nifti)
    
    output_pdf = output_pth+os.path.basename(confounds_file).replace('.tsv', '_sd-diff.pdf')
    f = plt.figure(figsize=(24,6))
    plot_stat_map(math_img('(infile.std(axis=-1)-cleaned_img.std(axis=-1))/infile.std(axis=-1)', infile=infile, cleaned_img=cleaned_img), 
                bg_img=mean_img, figure=f, cut_coords=(0,0,0), threshold=0.125, vmax=1, cmap='viridis', output_file=output_pdf)
    
    return output_pdf, output_nifti
    

In [32]:

# # STANDARD SCALING 1ST, THEN OBTAINING COMPONENTS
# # we assume the confounds nor the nifti_file need temporal filtering 
# from sklearn.preprocessing import StandardScaler
# stand_conf = StandardScaler().fit_transform(np.nan_to_num(confounds_table))

# from sklearn.decomposition import PCA
# pca = PCA(0.95,whiten=True) #choose the minimum number of principal components such that 95% of the variance is retained.
# pca_confs = pca.fit_transform(np.nan_to_num(stand_conf))
# num_comp = pca.n_components_

# print(pca.explained_variance_)
# print(pca.explained_variance_ratio_)
# print(pca.explained_variance_ratio_.cumsum())

# all_confs = pd.DataFrame(pca_confs, columns=['comp_{n}'.format(n=n) for n in range(num_comp)])
# print(all_confs)

In [33]:

# # NO SCALING, JUST OBTAINING COMPONENTS
# # we assume the confounds nor the nifti_file need temporal filtering 

# from sklearn.decomposition import PCA
# pca = PCA(0.95,whiten=True) #choose the minimum number of principal components such that 95% of the variance is retained.
# pca_confs = pca.fit_transform(np.nan_to_num(confounds_table))
# num_comp = pca.n_components_

# print(pca.explained_variance_)
# print(pca.explained_variance_ratio_)
# print(pca.explained_variance_ratio_.cumsum())

# all_confs = pd.DataFrame(pca_confs, columns=['comp_{n}'.format(n=n) for n in range(num_comp)])
# print(all_confs)

In [16]:
# Use confounds as nuisance regressors in a GLM
#define Node
confGLM = pe.MapNode(Function(input_names=['nifti_file', 'confounds_file', 'output_pth'],#'which_confounds','output_pth'], 
                              output_names=['output_pdf', 'output_nifti'],
                             function=nistats_confound_pca_glm),
                             name='nistats_confound_pca_glm', 
                             iterfield=["nifti_file", "confounds_file"])

In [17]:
# define settings
#confGLM.inputs.which_confounds = analysis_params['nuisance_columns']
confGLM.inputs.nifti_file = res_psc.outputs.out_file
confGLM.inputs.confounds_file = res_SG_filter_confounds.outputs.out
confGLM.inputs.output_pth = dest_pth+'sub-'+sub_num+'/ses-'+ses_num+'/func/'

In [18]:
# run and print outputs
res_confGLM = confGLM.run()

190123-15:07:15,585 nipype.workflow INFO:
	 [Node] Setting-up "nistats_confound_pca_glm" in "/tmp/tmptdq26ju5/nistats_confound_pca_glm".
190123-15:07:15,592 nipype.workflow INFO:
	 [Node] Setting-up "_nistats_confound_pca_glm0" in "/tmp/tmptdq26ju5/nistats_confound_pca_glm/mapflow/_nistats_confound_pca_glm0".
190123-15:07:15,596 nipype.workflow INFO:
	 [Node] Running "_nistats_confound_pca_glm0" ("nipype.interfaces.utility.wrappers.Function")


/opt/miniconda-latest/envs/neuro/lib/python3.6/site-packages/numpy/core/_methods.py:138: RuntimeWarning: invalid value encountered in sqrt
  ret = um.sqrt(ret, out=ret)
<string>:1: RuntimeWarning: invalid value encountered in true_divide


190123-15:07:42,929 nipype.workflow INFO:
	 [Node] Finished "_nistats_confound_pca_glm0".
190123-15:07:42,932 nipype.workflow INFO:
	 [Node] Setting-up "_nistats_confound_pca_glm1" in "/tmp/tmptdq26ju5/nistats_confound_pca_glm/mapflow/_nistats_confound_pca_glm1".
190123-15:07:42,935 nipype.workflow INFO:
	 [Node] Running "_nistats_confound_pca_glm1" ("nipype.interfaces.utility.wrappers.Function")


<string>:1: RuntimeWarning: invalid value encountered in true_divide


190123-15:08:10,434 nipype.workflow INFO:
	 [Node] Finished "_nistats_confound_pca_glm1".
190123-15:08:10,437 nipype.workflow INFO:
	 [Node] Setting-up "_nistats_confound_pca_glm2" in "/tmp/tmptdq26ju5/nistats_confound_pca_glm/mapflow/_nistats_confound_pca_glm2".
190123-15:08:10,440 nipype.workflow INFO:
	 [Node] Running "_nistats_confound_pca_glm2" ("nipype.interfaces.utility.wrappers.Function")


<string>:1: RuntimeWarning: invalid value encountered in true_divide


190123-15:08:37,340 nipype.workflow INFO:
	 [Node] Finished "_nistats_confound_pca_glm2".
190123-15:08:37,344 nipype.workflow INFO:
	 [Node] Finished "nistats_confound_pca_glm".


In [19]:
res_confGLM.outputs.output_nifti

['/home/neuro/projects/data/derivatives/post_fmriprep/sub-05/ses-01/func/sub-05_ses-01_task-soma_run-01_bold_space-T1w_preproc_sg_psc_nuis.nii.gz',
 '/home/neuro/projects/data/derivatives/post_fmriprep/sub-05/ses-01/func/sub-05_ses-01_task-soma_run-03_bold_space-T1w_preproc_sg_psc_nuis.nii.gz',
 '/home/neuro/projects/data/derivatives/post_fmriprep/sub-05/ses-01/func/sub-05_ses-01_task-soma_run-04_bold_space-T1w_preproc_sg_psc_nuis.nii.gz']

In [20]:
def leave_one_out_lists(input_list):
    """leave_one_out_lists takes creates a list of lists, with each element
    of the input_list left out of the returned lists once, in order.
    Parameters
    ----------
    input_list : list
        list of items, for instance absolute paths to nii files
    Returns
    -------
    output_data : list
        list of lists
    """

    out_lists = []
    for x in input_list:
        out_lists.append([y for y in input_list if y != x])

    return out_lists

In [39]:
leave_one_out_lists(res_confGLM.outputs.output_nifti)

[['/home/neuro/projects/data/derivatives/post_fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-soma_run-02_bold_space-T1w_preproc_sg_psc_nuis.nii.gz',
  '/home/neuro/projects/data/derivatives/post_fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-soma_run-03_bold_space-T1w_preproc_sg_psc_nuis.nii.gz',
  '/home/neuro/projects/data/derivatives/post_fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-soma_run-04_bold_space-T1w_preproc_sg_psc_nuis.nii.gz'],
 ['/home/neuro/projects/data/derivatives/post_fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-soma_run-01_bold_space-T1w_preproc_sg_psc_nuis.nii.gz',
  '/home/neuro/projects/data/derivatives/post_fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-soma_run-03_bold_space-T1w_preproc_sg_psc_nuis.nii.gz',
  '/home/neuro/projects/data/derivatives/post_fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-soma_run-04_bold_space-T1w_preproc_sg_psc_nuis.nii.gz'],
 ['/home/neuro/projects/data/derivatives/post_fmriprep/sub-02/ses-01/func/sub-02_ses-01_task-soma_run-01

In [38]:
# TRY TO LOOK AT SOMA DATA ############################

In [40]:
# somas = res_confGLM.outputs.output_nifti # list of residual niftis
# mean_filename = (res_confGLM.outputs.output_nifti[0]).replace('run-01', 'run-mean')

# mean_soma=image.math_img('(i1+i2+i3+i4)/4', i1=somas[0], i2=somas[1], i3=somas[2],i4=somas[3])
# mean_soma.to_filename(mean_filename)

In [71]:
# nr_stims = len(analysis_params['soma_stimulus']) #number of stim videos
# stimulus_setup = np.array([s.split('.avi')[0] for s in analysis_params['soma_stimulus']]) # name for all regressors
# different_regressors = np.unique(stimulus_setup)

In [78]:
# onset_times = np.arange(0, 2.75*nr_stims, 2.75) + 12
# onset_times

array([  12.  ,   14.75,   17.5 ,   20.25,   23.  ,   25.75,   28.5 ,
         31.25,   34.  ,   36.75,   39.5 ,   42.25,   45.  ,   47.75,
         50.5 ,   53.25,   56.  ,   58.75,   61.5 ,   64.25,   67.  ,
         69.75,   72.5 ,   75.25,   78.  ,   80.75,   83.5 ,   86.25,
         89.  ,   91.75,   94.5 ,   97.25,  100.  ,  102.75,  105.5 ,
        108.25,  111.  ,  113.75,  116.5 ,  119.25,  122.  ,  124.75,
        127.5 ,  130.25,  133.  ,  135.75,  138.5 ,  141.25,  144.  ,
        146.75,  149.5 ,  152.25,  155.  ,  157.75,  160.5 ,  163.25,
        166.  ,  168.75,  171.5 ,  174.25])

In [79]:
# {er: onset_times[stimulus_setup==er] for er in different_regressors}

{'bhand_fing1': array([  50.5,  144. ]),
 'bhand_fing2': array([  53.25,  141.25]),
 'bhand_fing3': array([  56. ,  138.5]),
 'bhand_fing4': array([  58.75,  135.75]),
 'bhand_fing5': array([  61.5,  133. ]),
 'bleg': array([  64.25,  146.75]),
 'eyebrows': array([  12.  ,   39.5 ,   75.25,   94.5 ,  122.  ,  157.75]),
 'eyes': array([  14.75,   42.25,   72.5 ,   97.25,  124.75,  155.  ]),
 'lhand_fing1': array([  23. ,  116.5]),
 'lhand_fing2': array([  25.75,  113.75]),
 'lhand_fing3': array([  28.5,  111. ]),
 'lhand_fing4': array([  31.25,  108.25]),
 'lhand_fing5': array([  34. ,  105.5]),
 'lleg': array([  36.75,  119.25]),
 'mouth': array([  17.5 ,   45.  ,   69.75,  100.  ,  127.5 ,  152.25]),
 'rhand_fing1': array([  78. ,  171.5]),
 'rhand_fing2': array([  80.75,  168.75]),
 'rhand_fing3': array([  83.5,  166. ]),
 'rhand_fing4': array([  86.25,  163.25]),
 'rhand_fing5': array([  89. ,  160.5]),
 'rleg': array([  91.75,  174.25]),
 'tongue': array([  20.25,   47.75,   67.  ,

In [80]:
# import pandas as pd
# stim_onset_list = []
# for i, s in enumerate(stimulus_setup):
#     if s[0] == 'b':
#         stim_onset_list.append([onset_times[i], 2.25, 'l'+s[1:]])
#         stim_onset_list.append([onset_times[i], 2.25, 'r'+s[1:]])        
#     stim_onset_list.append([onset_times[i], 2.25, s])
# events = pd.DataFrame(stim_onset_list, columns=['onset','duration','trial_type'])

60

In [57]:
141*1.6

225.60000000000002